In [546]:
### use generate_graph() to transform adjeceny matrix to graph
import numpy as np
from graph_tools import *
def generate_graph(AM):
    d = Graph(directed=False)
    d.add_vertex(len(AM)-1)
    for i in range(len(AM)):
        for j in range(i,len(AM)):
            if AM[i][j]==1:
                d.add_edge(i,j)
    return d

In [547]:
###get adjeceny matrix from user
def enter_adjMatrix():
    n = int(input("Enter number of rows(columns) in the matrix: "))
    matrix = []
    print("Enter the %s x %s matrix row by row: "% (n, n))
    for i in range(n):
        matrix.append(list(map(int, input().rstrip().split())))  
    return matrix

In [548]:
def composite_permutition(p,g):
    p1=Permutation(p)
    q1=Permutation(g)
    return [(p1*q1)(i) for i in range(p1.size)]


In [829]:
def apply_permute(AM,pi):
    l=[]
    l1=[]
    l2=[]
    l0=[]
    for i in range (len(AM)):
        for j in range (len(AM)):
            if AM[i][j]==1 :
                l.append([i,j])
    n=len(l)
    for i in range (n):
        l1.append( sorted(l[i]))
    l2=sorted(l1)
    for i in range (len(l2)-1):
        if l2[i]==l2[(i+1)]:
            l0.append(l2[i])
    l3=np.zeros([len(l0),2])
    for i in range (10): 
        for j in range (2):
            for k in range(10): 
                if l0[i][j]==k:
                    l3[i][j]=pi[k]
    AM0=np.zeros([len(AM),len(AM)])
    for i in range (len(l3)):
        a,b=l3[i]
        AM0[int(a),int(b)]= AM0[int(b),int(a)]=1
    return AM0


In [550]:
def tran_permu_to_graph(g):
    d = Graph(directed=False)
    d.add_vertex(len(g)-1)
    for i in range(len(g)):
        d.add_edge(g[(i)],g[(i+1)%len(g)])
    return d

In [551]:
### find the inverse of permutation using inv()
def inv(perm):
    per=list(perm)
    inverse = [0] * len(per)
    for i, p in enumerate(per):
        inverse[p] = i
    return inverse

In [553]:
def are_equal(AM,AM1) :
    return np.array_equal(AM1,AM)

In [554]:
def graph_isomorphism(P,V):
    message_list=[]
    prover_move=True
    while len(message_list)<4:
        if prover_move :
            message_list=P(message_list)
        else :
            message_list=V(message_list)
        prover_move= not prover_move
        if message_list[-1]== 'accept' or   message_list[-1]== 'reject':
            return message_list
    return message_list

In [555]:
import math
def honest_prover(AM,AM1,pi,mess_list,s):
    n=len(AM)
    if len(mess_list)==0:
        np.random.seed(s)
        sigma =np.random.permutation(n)
        h=apply_permute(AM,sigma)
        mess_list.append(h)
    if len(mess_list)==2:
        np.random.seed(s)
        sigma =np.random.permutation(n)
        ch=mess_list[1]
        if ch==0:
            mess_list.append(sigma)
        else:
            p=inv(pi)
            mess_list.append(composite_permutition(p,list(sigma)))
    return mess_list

In [556]:
def honest_verifier(AM,AM1,mess_list):
    if len(mess_list)==1:
        ch=random.choice(range(2))
        mess_list.append(ch)
    if len(mess_list)==3: 
        phi=mess_list[2]
        ch=mess_list[1]
        H=mess_list[0]
        mess_list[0]=generate_graph(H)
        if ch==0 :
            H0=apply_permute(AM,phi)
        if ch==1:
            H0=apply_permute(AM1,phi)
        if are_equal(H,H0):
            mess_list.append('accept')
        else:
            mess_list.append('reject')
    return mess_list

In [718]:
def test_isomorphism():
#     AM=enter_adjMatrix()
#     AM1=enter_adjMatrix()
#     pi=list(map(int, input('Enter pi:').split())) 
    s=random.choice(range(1000))
    V=lambda msg: honest_verifier(AM,AM1,msg)
    P=lambda msg: honest_prover(AM,AM1,pi,msg,s)
    return graph_isomorphism(P,V)

In [756]:
import csv
def get_graph_from_file(i):
    with open('matices.csv') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    l=[]
    l=content[i]
    matrix=[]
    l[i]
    n=int(np.sqrt(len(l)-2))
    for i in range(1,len(l)-1):
        matrix.append(l[i])
    s=np.reshape(matrix,(n,n))
    return s

In [755]:
def get_pi_from_file(i):
    with open('matices.csv') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    l=[]
    l=content[i]
    matrix=[]
    l[i]
    n=int(np.sqrt(len(l)-2))
    for i in range(1,len(l)-1):
        matrix.append(int(l[i]))
    return matrix

In [765]:
def equal(AM):
    a=np.zeros([len(AM),len(AM)])
    for i in range(len(AM)):
        for j in range(len(AM)):
            a[i][j]=AM[i,j]
    return a

In [865]:
M=get_graph_from_file(0)
AM=equal(M)
M1=get_graph_from_file(1)
AM1=equal(M1)
pi=get_pi_from_file(2)

In [732]:
def cheating_prover(AM, AM1, mess_list,s):
    n=len(AM)
    if len(mess_list)==0:
        np.random.seed(s)
        sigma =np.random.permutation(n)
        h=apply_permute(AM,sigma)
        mess_list.append(h)
    if len(mess_list)==2:
        np.random.seed(s)
        sigma =np.random.permutation(n)
        ch=mess_list[1]
        if ch==0:
            mess_list.append(sigma)
        else:
            mess_list.append(np.random.permutation(n))
    return mess_list

In [730]:
def protocol_dishonest_prover():
    V=lambda msg: honest_verifier(AM,AM1,msg)
    P=lambda msg: cheating_prover(AM,AM1,msg,s)
    return graph_isomorphism(P,V)

In [868]:
M1=get_graph_from_file(3)
AM1=equal(M1)

##  Example between honest prover and honest verifier when ch=0

In [723]:
test_isomorphism()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


[// Generated by graph-tools (version 1.0) at 2020/53/05/07/20 02:53:36
 // undirected, 4 vertices, 4 edges
 graph export_dot {
   node [color=gray90,style=filled];
   "0";
   "1";
   "2";
   "3";
   "0" -- "1";
   "0" -- "2";
   "1" -- "3";
   "2" -- "3";
 }, 0, array([1, 0, 2, 3]), 'accept']

##  Example between honest prover and honest verifier when ch=1

In [724]:
test_isomorphism()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


[// Generated by graph-tools (version 1.0) at 2020/53/05/07/20 02:53:57
 // undirected, 4 vertices, 4 edges
 graph export_dot {
   node [color=gray90,style=filled];
   "0";
   "1";
   "2";
   "3";
   "0" -- "1";
   "0" -- "2";
   "1" -- "3";
   "2" -- "3";
 }, 1, [1, 2, 3, 0], 'accept']

##  Example between cheating prover and honest verifier when ch=0

In [733]:
protocol_dishonest_prover()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


[// Generated by graph-tools (version 1.0) at 2020/21/05/07/20 03:21:07
 // undirected, 4 vertices, 4 edges
 graph export_dot {
   node [color=gray90,style=filled];
   "0";
   "1";
   "2";
   "3";
   "0" -- "1";
   "0" -- "2";
   "1" -- "3";
   "2" -- "3";
 }, 0, array([2, 3, 1, 0]), 'accept']

##  Example between cheating prover and honest verifier when ch=1

In [736]:
protocol_dishonest_prover()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


[// Generated by graph-tools (version 1.0) at 2020/21/05/07/20 03:21:29
 // undirected, 4 vertices, 4 edges
 graph export_dot {
   node [color=gray90,style=filled];
   "0";
   "1";
   "2";
   "3";
   "0" -- "1";
   "0" -- "2";
   "1" -- "3";
   "2" -- "3";
 }, 1, array([0, 1, 3, 2]), 'reject']

In [845]:
M=get_graph_from_file(4)
AM=equal(M)
M1=get_graph_from_file(5)
AM1=equal(M1)
pi=get_pi_from_file(6)

In [864]:
AM=[[0, 1, 1, 1, 1, 1, 1, 1, 0, 1], 
[1, 0, 1, 1, 1, 1, 0, 1, 1, 0], 
[1, 1, 0, 1, 1, 1, 0, 0, 1, 0], 
[1, 1, 1, 0, 1, 1, 0, 0, 0, 0], 
[1, 1, 1, 1, 0, 1, 0, 0, 0, 1], 
[1, 1, 1, 1, 1, 0, 1, 0, 1, 1], 
[1, 0, 0, 0, 0, 1, 0, 1, 0, 1], 
[1, 1, 0, 0, 0, 0, 1, 0, 1, 0], 
[0, 1, 1, 0, 0, 1, 0, 1, 0, 0], 
[1, 0, 0, 0, 1, 1, 1, 0, 0, 0]] 
pi=[8, 1, 5, 0, 7, 2, 9, 4, 3, 6]
AM1=apply_permute(AM,pi)
apply_permute(AM,[2, 0, 9, 5, 8, 7, 1, 4, 3, 6])

array([[1., 0., 1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [862]:
AM1

array([[1., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 1., 1., 0., 1., 1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [843]:
with open('matices.csv') as f:
        content = f.readlines()
content = [x.strip() for x in content] 
l=[]
l=content[4]
len(l)

102

In [854]:
test_isomorphism()


/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


[// Generated by graph-tools (version 1.0) at 2020/34/05/07/20 05:34:39
 // undirected, 9 vertices, 11 edges
 graph export_dot {
   node [color=gray90,style=filled];
   "0";
   "1";
   "2";
   "4";
   "5";
   "6";
   "7";
   "8";
   "9";
   "0" -- "0";
   "0" -- "2";
   "0" -- "5";
   "0" -- "9";
   "1" -- "2";
   "2" -- "4";
   "2" -- "5";
   "2" -- "6";
   "2" -- "7";
   "2" -- "8";
   "2" -- "9";
 }, 0, array([2, 0, 9, 5, 8, 7, 1, 4, 3, 6]), 'accept']

In [863]:
test_isomorphism()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


[// Generated by graph-tools (version 1.0) at 2020/40/05/07/20 05:40:19
 // undirected, 9 vertices, 11 edges
 graph export_dot {
   node [color=gray90,style=filled];
   "0";
   "1";
   "3";
   "4";
   "5";
   "6";
   "7";
   "8";
   "9";
   "0" -- "0";
   "0" -- "6";
   "1" -- "4";
   "1" -- "6";
   "3" -- "6";
   "4" -- "6";
   "4" -- "9";
   "5" -- "6";
   "6" -- "7";
   "6" -- "8";
   "6" -- "9";
 }, 1, [1, 4, 0, 2, 3, 9, 7, 5, 6, 8], 'reject']

In [828]:
# AM=np.array([[0,1,0,1],[1,0,1,0],[0,1,0,1],[1,0,1,0]])
# pi=[0,3,1,2]
apply_permutei(AM,pi)

22


array([[0., 1., 0., 0., 0., 1., 1., 1., 0., 0.],
       [1., 0., 0., 0., 1., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 1., 1.],
       [1., 1., 1., 1., 1., 0., 0., 0., 1., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 1., 1., 0., 0., 1.],
       [0., 1., 1., 0., 1., 0., 1., 0., 1., 0.]])

In [866]:
def simulator(AM,AM1,s):
    mess_list=[]
    sigma =np.random.seed(s)
    sigma =np.random.permutation(n)
    h=apply_permut(AM,sigma)
    mess_list.append(h)
    b=random.choice(range(2))
    ch=random.choice(range(2))
    if ch == b:
        print('accept')
        return sigma
    else :
        simulator(AM,AM1,s)

In [869]:
simulator(AM,AM1,2)

accept


array([2, 3, 1, 0])